In [1]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd

CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'

Gtk-Message: 08:43:49.786: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/D-visualize/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/D-visualize/data/pymol/icons/icon2.svg', because: No such file or directory


Could not read PyMOL stylesheet.
DEBUG: PYMOL_DATA='./data'
 Detected OpenGL version 4.6. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.60.
PyMOL>set fetch_path, /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files
 Setting: fetch_path set to /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files.
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
135.60927
0 / 1 pockets found. [135.60927]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
70.11315
0 / 1 pockets found. [70.11315]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
84.67005
44.810974
0 / 1 pockets found. [44.810974]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
44.97899
59.600872
0 / 1 pockets found. [44.97899]
 S

In [ ]:
import sys
import numpy as np
sys.path.append('/home/vit/projects/cryptoshow-analysis/src/utils')
import eval_utils
import cryptoshow_utils
import vis_utils

CBS_DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/cryptobench-clustered-binding-sites.txt'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions'

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=CBS_DATA_PATH)
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
p2rank_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/p2rank', protein_ids=binding_residues_mmcifed.keys())
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/finetuning-with-smoothing', protein_ids=binding_residues_mmcifed.keys())

# map binding residues to auth labeling
binding_residues = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in binding_residues_mmcifed.items()}
p2rank_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in p2rank_predictions_mmcifed.items()}
model_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/vit/.local/lib/python3.11/site-packages/biotite/structure/io/pdbx/cif.py", line 658, in __getitem__
    category = CIFCategory.deserialize(category, expect_whitespace)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vit/.local/lib/python3.11/site-packages/biotite/structure/io/pdbx/cif.py", line 382, in deserialize
    category_dict = CIFCategory._deserialize_looped(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vit/.local/lib/python3.11/site-packages/biotite/structure/io/pdbx/cif.py", line -1, in _deserialize_looped
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/vit/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_32989/794419337.py", line 21, in <module>
    binding_residues

In [ ]:
COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/coordinates'

avg_success_pocket_size = []
for protein_id in binding_residues.keys(): 
    this_binding_residues = binding_residues_mmcifed[protein_id]
    this_p2rank_predictions = p2rank_predictions_mmcifed[protein_id]
    sys.path.append('/home/skrhakv/cryptoshow-analysis/src/utils')
    sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/utils')
    import cryptoshow_utils
    
    
    for true_pocket in this_binding_residues:
        dcc = float('inf')
        coordinates = np.load(f'{COORDINATES_DIR}/{protein_id.replace("_", "")}.npy')
        for predicted_pocket in this_p2rank_predictions:
            this_dcc = cryptoshow_utils.compute_center_distance(coordinates, true_pocket, predicted_pocket)
            dcc = min(dcc, this_dcc)
        if dcc < 4.0:
            avg_success_pocket_size.append(len(true_pocket))

np.median(avg_success_pocket_size)

23.0

In [ ]:
for protein_id in binding_residues.keys():
    this_p2rank_predictions = np.concatenate(p2rank_predictions[protein_id]) if p2rank_predictions[protein_id] else np.array([])
    this_model_predictions = np.concatenate(model_predictions[protein_id]) if model_predictions[protein_id] else np.array([])
    this_binding_residues = np.concatenate(binding_residues[protein_id]) if binding_residues[protein_id] else np.array([])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    if len(this_binding_residues) != 0:
        cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, this_binding_residues))
    else:
        continue
    
    if len(this_model_predictions) != 0:
        cmd.color('forest', vis_utils.generate_pymol_algebra_selection(protein_id, this_model_predictions))
        intersection1 = vis_utils.get_intersection(this_binding_residues, this_model_predictions)
        if len(intersection1) != 0:
            cmd.color('lime', vis_utils.generate_pymol_algebra_selection(protein_id, intersection1))


    if len(this_p2rank_predictions) != 0:
        cmd.color('blue', vis_utils.generate_pymol_algebra_selection(protein_id, this_p2rank_predictions))
        intersection2 = vis_utils.get_intersection(this_binding_residues, this_p2rank_predictions)
        if len(intersection2) != 0:
            cmd.color('cyan', vis_utils.generate_pymol_algebra_selection(protein_id, intersection2))

        intersection_of_all = vis_utils.get_intersection(intersection1, intersection2)
        if len(intersection_of_all) != 0:
            cmd.color('yellow', vis_utils.generate_pymol_algebra_selection(protein_id, intersection_of_all))

    DCCs = vis_utils.compute_DCCs(protein_id, binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id])
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')
    
    merged_predictions = p2rank_predictions_mmcifed[protein_id] + model_predictions_mmcifed[protein_id]
    vis_utils.draw_pocket_centers(binding_residues_mmcifed[protein_id], merged_predictions, protein_id)
    vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], merged_predictions, protein_id)

    # cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
20.998554
19.656574
21.088614
14.021774
41.05787
0 / 1 pockets found. [14.021774]


In [ ]:
COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/coordinates'
def draw_pocket_centers(binding_residues, predicted_residues, protein_id, color='blue'):
    from pymol import cmd
    from pymol.cgo import COLOR, SPHERE
    coordinates = np.load(f'{COORDINATES_DIR}/{protein_id.replace("_", "")}.npy')
    for i, pocket in enumerate(binding_residues):
        center = coordinates[pocket].mean(axis=0)
        spherelist = [
           COLOR,    1.000,    0.000,    0.000,
           SPHERE,   center[0],   center[1],   center[2], 0.30,
            ]
        cmd.load_cgo(spherelist, f'true_pocket_{i}', 1)
        cmd.pseudoatom(f'true_pocket_center_{i}', pos=center.tolist())

    for i, pocket in enumerate(predicted_residues):
        center = coordinates[pocket].mean(axis=0)
        if color == 'green':
            spherelist = [
               COLOR,    0.000,    1.000,    0.000,
               SPHERE,   center[0],   center[1],   center[2], 0.30,
                ]
        else:
            spherelist = [
               COLOR,    0.000,    0.000,    1.000,
               SPHERE,   center[0],   center[1],   center[2], 0.30,
                ]
        cmd.load_cgo(spherelist, f'predicted_pocket_{i}', 1)
        cmd.pseudoatom(f'predicted_pocket_center_{i}_{color}', pos=center.tolist())
        for j, true_pocket in enumerate(binding_residues):
            cmd.distance(f'distance_{i}_{j}', f'true_pocket_center_{j}', f'predicted_pocket_center_{i}_{color}')

for protein_id in binding_residues.keys():
    this_p2rank_predictions = np.concatenate(p2rank_predictions[protein_id]) if p2rank_predictions[protein_id] else np.array([])
    this_model_predictions = np.concatenate(model_predictions[protein_id]) if model_predictions[protein_id] else np.array([])
    this_binding_residues = np.concatenate(binding_residues[protein_id]) if binding_residues[protein_id] else np.array([])
    
    merged_predictions = p2rank_predictions_mmcifed[protein_id] + model_predictions_mmcifed[protein_id]
    DCCs = vis_utils.compute_DCCs(protein_id, binding_residues_mmcifed[protein_id], merged_predictions)
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')

    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    if len(this_binding_residues) != 0:
        cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, this_binding_residues))
    else:
        continue
    
    if len(this_model_predictions) != 0:
        cmd.color('forest', vis_utils.generate_pymol_algebra_selection(protein_id, this_model_predictions))
        intersection1 = vis_utils.get_intersection(this_binding_residues, this_model_predictions)
        if len(intersection1) != 0:
            cmd.color('lime', vis_utils.generate_pymol_algebra_selection(protein_id, intersection1))


    if len(this_p2rank_predictions) != 0:
        cmd.color('blue', vis_utils.generate_pymol_algebra_selection(protein_id, this_p2rank_predictions))
        intersection2 = vis_utils.get_intersection(this_binding_residues, this_p2rank_predictions)
        if len(intersection2) != 0:
            cmd.color('cyan', vis_utils.generate_pymol_algebra_selection(protein_id, intersection2))

        intersection_of_all = vis_utils.get_intersection(intersection1, intersection2)
        if len(intersection_of_all) != 0:
            cmd.color('yellow', vis_utils.generate_pymol_algebra_selection(protein_id, intersection_of_all))
    
    draw_pocket_centers(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)
    draw_pocket_centers(binding_residues_mmcifed[protein_id], model_predictions_mmcifed[protein_id], protein_id, color='green')
    # vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)
    # vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], model_predictions_mmcifed[protein_id], protein_id)

    # cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

20.998554
19.656574
21.088614
14.021774
41.05787
21.980396
22.402721
18.067108
20.223907
22.745678
0 / 1 pockets found. [14.021774]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
7.8622446
3.473199
15.464708
18.02419
23.794159
15.6300745
12.774979
5.4894896
1 / 1 pockets found. [3.473199]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
10.244024
16.52279
14.475388
14.372983
16.831547
0 / 1 pockets found. [10.244024]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
PyMOL>show surface
4.0950685
21.793299
18.812605
8.967337
11.430943
0 / 1 pockets found. [4.0950685]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF


# Show KMEANS/GMM
Show how KMEANS/GMM behaves as a clustering algorithm.

In [ ]:
import sys
import numpy as np
sys.path.append('/home/vit/projects/cryptoshow-analysis/src/utils')
import eval_utils
import cryptoshow_utils
import vis_utils

CBS_DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/cryptobench-clustered-binding-sites.txt'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/D-visualize/predictions'

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=CBS_DATA_PATH)
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/GMMspherical-finetuning-with-smoothing', protein_ids=binding_residues_mmcifed.keys())
model_predictions_mmcifed = {protein_id: [[pocket] for pocket, _ in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}
# map binding residues to auth labeling
model_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket[0]) for pocket in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}
binding_residues = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket) for pocket in pockets] for protein_id, pockets in binding_residues_mmcifed.items()}


### Per pocket visualization
Show each pocket in different color.

#### NOTE - KMEANS
It seems that KMEANS doesn't work nicely. There are two main problems:
1. sometimes the clusters are blended into each other
2. it doesn't take distance into the account. So sometimes the pocket cluster has residues that are far away.

#### NOTE - GMM tied, same for full
1. sometimes clsuters blended to each other
2. also, distant sites clustered together

#### GMM spherical
Makes the most sense visually, although also not perfect

In [16]:
COLORS = ['pink', 'red', 'blue', 'green', 'brown', 'forest', 'sand', 'skyblue', 'slate', 'smudge', 'splitpea', 'sulfur', 'teal', 'tv_blue', 'tv_green', 'tv_orange', 'tv_red', 'tv_yellow']

for protein_id in binding_residues.keys():
    this_model_predictions = np.concatenate(model_predictions[protein_id]) if model_predictions[protein_id] else np.array([])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    

    for i, pocket in enumerate(model_predictions[protein_id]):
        cmd.color(COLORS[i % len(COLORS)], vis_utils.generate_pymol_algebra_selection(protein_id, pocket))
    # if len(this_model_predictions) != 0:
    #     cmd.color('forest', vis_utils.generate_pymol_algebra_selection(protein_id, this_model_predictions))
    #     intersection1 = vis_utils.get_intersection(this_binding_residues, this_model_predictions)
    #     if len(intersection1) != 0:
    #         cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, intersection1))

    DCCs = vis_utils.compute_DCCs(protein_id, binding_residues_mmcifed[protein_id], model_predictions_mmcifed[protein_id])
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')
    
    # merged_predictions = model_predictions_mmcifed[protein_id]
    # vis_utils.draw_pocket_centers(binding_residues_mmcifed[protein_id], merged_predictions, protein_id)
    # vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], merged_predictions, protein_id)

    cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break